# Clustering Crypto

In [29]:
pip install path

In [30]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans




### Deliverable 1: Preprocessing the Data for PCA

In [69]:
# Load the crypto_data.csv dataset.
file_path = "<Challenge 19>/crypto_data.csv"
crypto_df = pd.read_csv("crypto_data.csv")
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [102]:
# Keep all the cryptocurrencies that are being traded.
crypto_df[crypto_df["IsTrading"] == True]


,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...,...
1243,SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
1244,UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
1245,BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
1246,ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [71]:
# Remove the "IsTrading" column. 
Is_remove = crypto_df
Is_remove.drop(columns=["IsTrading"], inplace=True)
Is_remove.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,PoW,NaN,611000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [72]:
# Create a new DataFrame that holds only the cryptocurrencies names.
cryptocurrencies_names = crypto_df["CoinName"]
cryptocurrencies_names

0              42 Coin
1              365Coin
2              404Coin
3            SixEleven
4                  808
             ...      
1247       BitcoinPlus
1248        DivotyCoin
1249       Giotto Coin
1250    OpenSourceCoin
1251         SteamPunk
Name: CoinName, Length: 1252, dtype: object

In [73]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(columns="CoinName", inplace=True)

In [74]:
# Keep the rows where coins are mined.
crypto_df[crypto_df["TotalCoinsMined"] > 0]

,Unnamed: 0,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,Scrypt,PoW/PoS,4.199995e+01,42
2,404,Scrypt,PoW/PoS,1.055185e+09,532000000
5,1337,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,SHA-256,PoW,1.792718e+07,21000000
8,ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
1242,GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,BDX,CryptoNight,PoW,9.802226e+08,1400222610
1246,ZEN,Equihash,PoW,7.296538e+06,21000000
1247,XBC,Scrypt,PoS,1.283270e+05,1000000


In [75]:
# Remove rows that have at least 1 null value.
df_crypto = crypto_df.dropna()
df_crypto

,Unnamed: 0,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,Scrypt,PoW/PoS,4.199995e+01,42
2,404,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
5,1337,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
1242,GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,BDX,CryptoNight,PoW,9.802226e+08,1400222610
1246,ZEN,Equihash,PoW,7.296538e+06,21000000
1247,XBC,Scrypt,PoS,1.283270e+05,1000000


In [76]:
df_crypto.dtypes

Unnamed: 0          object
Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [77]:
# convert TotalCoinSupply from object/str to number
df_crypto['TotalCoinSupply'] = df_crypto['TotalCoinSupply'].astype('float')

C:\Users\jingw\AppData\Local\Temp\ipykernel_2544\2033487494.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [78]:
df_crypto = df_crypto.iloc[:,1:]

In [79]:
# Use get_dummies() to create variables for text features.
crypto_df = pd.get_dummies(df_crypto, prefix=None, columns=None, drop_first=False)

In [80]:
# Standardize the data with StandardScaler().
crypto_standarize = StandardScaler().fit_transform(crypto_df)
print(crypto_standarize[0:5])  

[[-0.10047714 -0.03668886 -0.03668644 -0.03668644 -0.03668644 -0.03668644
  -0.05191741 -0.09016696 -0.03668644 -0.05191741 -0.05191741 -0.03668644
  -0.03668644 -0.18257419 -0.05191741 -0.03668644 -0.03668644 -0.08225509
  -0.03668644 -0.0974575  -0.06362848 -0.03668644 -0.03668644 -0.16620562
  -0.03668644 -0.03668644 -0.13848495 -0.03668644 -0.03668644 -0.07352146
  -0.05191741 -0.03668644 -0.03668644 -0.03668644 -0.06362848 -0.03668644
  -0.07352146 -0.09016696 -0.09016696 -0.03668644 -0.03668644 -0.12250233
  -0.12803688 -0.15291752 -0.03668644 -0.08225509 -0.03668644 -0.03668644
  -0.06362848 -0.16188544 -0.03668644 -0.03668644 -0.03668644 -0.07352146
  -0.1786061  -0.32732684 -0.03668644 -0.0974575  -0.08225509 -0.05191741
  -0.03668644  1.42714214 -0.06362848 -0.03668644 -0.03668644 -0.03668644
  -0.08225509 -0.06362848 -0.03668644 -0.03668644 -0.03668644 -0.03668644
  -0.05191741 -0.03668644 -0.03668644 -0.40984739 -0.03668644 -0.18257419
  -0.03668644 -0.10425721 -0.07352146 

### Deliverable 2: Reducing Data Dimensions Using PCA

In [81]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components = 3)
pca.fit(crypto_standarize)

PCA(n_components=3)

In [82]:
# Create a DataFrame with the three principal components.
data_pca = pca.transform(crypto_df)
data_pca = pd.DataFrame(data_pca,columns=['PC1','PC2','PC3'])
data_pca.head()

C:\Users\jingw\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning:

X has feature names, but PCA was fitted without feature names



,PC1,PC2,PC3
0,2.226891e+01,2.935562e-01,-9.659340e-01
1,5.670686e+08,-2.856177e+07,-1.340246e+07
2,-6.859409e-02,4.077875e-01,-1.288585e-01
3,1.315051e+10,1.177889e+10,-3.786729e+09
4,9.531012e+06,1.699995e+04,-3.641322e+05


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [83]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_df)
    inertia.append(km.inertia_)
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

C:\Users\jingw\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1036: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [84]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=5)
model

# Fit the model
model.fit(crypto_df)

# Predict clusters
predictions = model.predict(crypto_df)
print(predictions)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [85]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
crypto_df["predictions"] = predictions 
crypto_df

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,...,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW,predictions
0,4.199995e+01,4.200000e+01,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.055185e+09,5.320000e+08,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2.927942e+10,3.141593e+11,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1.792718e+07,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1242,1.493105e+07,2.500000e+08,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1245,9.802226e+08,1.400223e+09,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1246,7.296538e+06,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1247,1.283270e+05,1.000000e+06,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
clustered_df = crypto_df

In [87]:
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] = cryptocurrencies_names

In [88]:
#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["class"] = model.labels_
clustered_df.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,...,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW,predictions,CoinName,class
0,4.199995e+01,4.200000e+01,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,42 Coin,0
2,1.055185e+09,5.320000e+08,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,404Coin,0
4,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,808,0
5,2.927942e+10,3.141593e+11,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,EliteCoin,0
7,1.792718e+07,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Bitcoin,0


In [89]:
# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(744, 116)


,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,...,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW,predictions,CoinName,class
0,4.199995e+01,4.200000e+01,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,42 Coin,0
2,1.055185e+09,5.320000e+08,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,404Coin,0
4,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,808,0
5,2.927942e+10,3.141593e+11,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,EliteCoin,0
7,1.792718e+07,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Bitcoin,0
8,1.076842e+08,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Ethereum,0
9,6.303924e+07,8.400000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Litecoin,0
10,9.031294e+06,2.200000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Dash,0
11,1.720114e+07,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Monero,0
12,1.133597e+08,2.100000e+08,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Ethereum Classic,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [90]:
data_pca["class"] = model.labels_

In [91]:
# Creating a 3D-Scatter with the PCA data and the clusters
df = data_pca
fig = px.scatter_3d(df, x='PC1', y='PC2', z='PC3',
              color='class')
fig.show()

In [96]:
# Create a table with tradable cryptocurrencies.
crypto_df = pd.read_csv("crypto_data.csv")
crypto_df[crypto_df["IsTrading"] == True].hvplot.table() 

:Table   [Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply]

In [106]:
# Print the total number of tradable cryptocurrencies.
sum(crypto_df["IsTrading"] == True)


1144

In [21]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = clustered_df.copy()

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
clustered_df["CoinName"] = plot_df

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
clustered_df["Class"] = plot_df

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,1
ETH,0.000000e+00,0.000109,Ethereum,1
LTC,8.400000e-05,0.000064,Litecoin,1
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,1
ETC,2.100000e-04,0.000115,Ethereum Classic,1
ZEC,2.100000e-05,0.000007,ZCash,1


In [22]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
df_shopping.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply")


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)